# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

def bar():
    print("Hello World!")

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()